# Feature Engineering

In [47]:
import pandas as pd
import numpy as np

## Import each table

In [48]:
hotels = pd.read_csv('../data/hotels_new.csv')
users = pd.read_csv('../data/users.csv')
flights = pd.read_csv('../data/flights.csv')
sessions = pd.read_csv('../data/sessions.csv')

# Create new interesting columns ON THE TABLES

In [49]:
hotels['price'] = hotels['rooms']*hotels['hotel_per_room_usd']*hotels['nights']
flights['price'] = flights['seats']*flights['base_fare_usd']

In [50]:
sessions['session_start'] = pd.to_datetime(sessions['session_start'])
sessions['session_end'] = pd.to_datetime(sessions['session_end'])
# Duration of each session
sessions['session_duration'] = sessions['session_end'] - sessions['session_start']

In [51]:
flights['departure_time'] = pd.to_datetime(flights['departure_time'])
flights['return_time'] = pd.to_datetime(flights['return_time'])

# Features from sessions

In [52]:
# Amount of trips per user (feature 1)
amount_of_trips = sessions[['trip_id', 'user_id']].groupby('user_id').nunique()
amount_of_trips.columns = ['amount_of_trips']


In [53]:
# Amount of canceled trips per user (feature 2)
canceled_trips_count = sessions[sessions['cancellation']][['trip_id','user_id']].groupby('user_id').nunique()
canceled_trips_count.columns = ['canceled_trips']


In [54]:
# Trip id of canceled trips
canceled_trip_ids = sessions[sessions['cancellation']]['trip_id'].unique()

In [55]:
# Sessions of trips that actually happened
sessions_valid = sessions[~sessions['trip_id'].isin(canceled_trip_ids)]

In [56]:
# Average session duration of trips that actually happened (feature 3)
valid_session_duration = sessions_valid.groupby('user_id')['session_duration'].mean()

In [57]:
#Average flight discount per user (feature 4)
flight_discount_amount = sessions_valid.groupby('user_id')['flight_discount_amount'].mean()
flight_discount_amount.fillna(0, inplace=True)

In [58]:
#Average hotel discount per user (feature 5)
hotel_discount_amount = sessions_valid.groupby('user_id')['hotel_discount_amount'].mean()
hotel_discount_amount.fillna(0, inplace=True)

# Features from hotels

In [59]:
# Df to relate user_id with trip_id (used in hotels and flights)
user_trip_id = sessions_valid[['user_id', 'trip_id']].drop_duplicates()
user_trip_id

,user_id,trip_id
0,536782,NaN
1,570600,NaN
2,511055,511055-57ed6923358d47a0a3be86eaf8128ece
3,677209,NaN
4,546867,NaN
...,...,...
47429,525089,525089-118e10d811ad440a8690feec4bff1f2a
47430,560497,560497-deba8e593f6a4fa8b94b75ee4a8f4ecd
47431,582801,582801-36ffbf38bbb944f29a14f0bba8b43186
47433,515523,515523-5fbd163e4452409395e1f4615bd29dd9


In [60]:
hotels = pd.merge(hotels,user_trip_id, on='trip_id', how='left')
hotels

,trip_id,hotel_name,nights,rooms,check_in_time,check_out_time,hotel_per_room_usd,check_in_dayofyear,check_in_hour,check_out_dayofyear,check_out_hour,price,user_id
0,582979-5475b9940ee240bf9328a0cc0bc9900c,NH Hotel - ottawa,7,1,2023-03-31 13:32:21.030,2023-04-07 11:00:00,110,90,13,97,11,770,582979.0
1,588386-233db0f627cf4aaaaeaa9a632daa5ec4,Accor - philadelphia,1,1,2023-04-04 10:43:44.175,2023-04-05 11:00:00,126,94,10,95,11,126,588386.0
2,514785-863ac3b4eb984f1eb226d67955623a5d,Choice Hotels - montreal,4,1,2023-06-21 20:06:24.300,2023-06-25 11:00:00,87,172,20,176,11,348,514785.0
3,600395-79e934e3dcf2462587a0beffbebd6783,Marriott - memphis,2,2,2023-04-21 15:58:28.200,2023-04-23 11:00:00,153,111,15,113,11,612,600395.0
4,474120-a87f749cf54b4f4ca9262d07080a06b2,Accor - san francisco,5,2,2023-02-02 14:18:11.205,2023-02-07 11:00:00,346,33,14,38,11,3460,474120.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13767,583623-70097bf1d1334cfb891ac2bd09f448ae,Shangri-La - montreal,4,1,2023-02-23 17:25:07.185,2023-02-27 11:00:00,142,54,17,58,11,568,583623.0
13768,512377-acf089357a6b497d91eda34bb532578d,Marriott - washington,6,1,2023-04-02 12:25:51.465,2023-04-08 11:00:00,446,92,12,98,11,2676,NaN
13769,497042-fa163de72e234eaf9c9bc8b477fbf66a,Hyatt - san antonio,2,1,2023-04-13 11:40:58.350,2023-04-15 11:00:00,103,103,11,105,11,206,497042.0
13770,598533-40848e5db09e46aa8f56c0c3ccdefb2d,Hyatt - macau,12,1,2024-01-24 09:54:31.635,2024-02-05 11:00:00,114,24,9,36,11,1368,NaN


In [61]:
hotels_valid = hotels[~hotels['trip_id'].isin(canceled_trip_ids)].copy()

In [62]:
hotels_valid[hotels_valid.isna().any(axis=1)]

,trip_id,hotel_name,nights,rooms,check_in_time,check_out_time,hotel_per_room_usd,check_in_dayofyear,check_in_hour,check_out_dayofyear,check_out_hour,price,user_id


In [63]:
# Average rooms per user (feature 6)
user_rooms = hotels_valid.groupby('user_id')['rooms'].mean()
# Average nights per user (feature 7)
user_nights = hotels_valid.groupby('user_id')['nights'].mean()
# average price per user (feature 8)
user_price = hotels_valid.groupby('user_id')['price'].mean()
user_hotel_price = user_price.rename('hotel_price')
# amount of trips with hotels per user (feature 9)
user_hotels = hotels_valid.groupby('user_id')['trip_id'].count()
user_hotels = user_hotels.rename('hotel_count')

In [64]:
flights = pd.merge(flights, user_trip_id, on='trip_id', how='left')
flights_valid = flights[~flights['trip_id'].isin(canceled_trip_ids)]

In [65]:
# Average flight seats per user (feature 10)
user_seats = flights_valid.groupby('user_id')['seats'].mean()
# Average flight price per user (feature 11)
user_flight_price = flights_valid.groupby('user_id')['price'].mean()
user_flight_price = user_flight_price.rename('flight_price')
# Amount of flights per user (feature 12)
user_flights = flights_valid.groupby('user_id')['trip_id'].count()
user_flights = user_flights.rename('flight_count')
# Amount of checked bags per user (feature 13)
user_checked_bags = flights_valid.groupby('user_id')['checked_bags'].mean()

# Features from users

In [66]:
users

,user_id,birthdate,gender,married,has_children,home_country,home_city,home_airport,home_airport_lat,home_airport_lon,sign_up_date
0,531931,1984-05-26,F,False,False,usa,san jose,SJC,37.362,-121.929,2023-01-15
1,611065,1981-08-20,F,False,False,usa,los angeles,LAX,33.942,-118.408,2023-02-22
2,586762,1987-09-26,F,False,False,canada,montreal,YUL,45.468,-73.741,2023-02-11
3,229330,1976-11-08,F,True,False,usa,colorado springs,COS,38.806,-104.700,2022-06-27
4,536035,1995-11-09,M,False,False,usa,new york,LGA,40.777,-73.872,2023-01-17
...,...,...,...,...,...,...,...,...,...,...,...
5777,591155,1982-12-26,M,True,False,canada,windsor,YQG,42.276,-82.956,2023-02-13
5778,521235,1989-06-21,F,False,False,usa,seattle,SEA,47.449,-122.309,2023-01-10
5779,600351,1976-07-15,F,True,False,usa,portland,PDX,45.589,-122.597,2023-02-17
5780,577044,1969-07-26,F,True,True,usa,chicago,UGN,42.422,-87.868,2023-02-06


In [67]:
users['birthdate'] = pd.to_datetime(users['birthdate'])
today = pd.Timestamp.today()
users['age'] = (today-users['birthdate']).dt.days//365
#users['birthday_dayofyear'] = users['birthdate'].dt.dayofyear
users['tenure_months'] = ((today-pd.to_datetime(users['sign_up_date'])).dt.days//30.41).astype('int64')

In [68]:
users

,user_id,birthdate,gender,married,has_children,home_country,home_city,home_airport,home_airport_lat,home_airport_lon,sign_up_date,age,tenure_months
0,531931,1984-05-26,F,False,False,usa,san jose,SJC,37.362,-121.929,2023-01-15,40,27
1,611065,1981-08-20,F,False,False,usa,los angeles,LAX,33.942,-118.408,2023-02-22,43,25
2,586762,1987-09-26,F,False,False,canada,montreal,YUL,45.468,-73.741,2023-02-11,37,26
3,229330,1976-11-08,F,True,False,usa,colorado springs,COS,38.806,-104.700,2022-06-27,48,33
4,536035,1995-11-09,M,False,False,usa,new york,LGA,40.777,-73.872,2023-01-17,29,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5777,591155,1982-12-26,M,True,False,canada,windsor,YQG,42.276,-82.956,2023-02-13,42,26
5778,521235,1989-06-21,F,False,False,usa,seattle,SEA,47.449,-122.309,2023-01-10,35,27
5779,600351,1976-07-15,F,True,False,usa,portland,PDX,45.589,-122.597,2023-02-17,48,26
5780,577044,1969-07-26,F,True,True,usa,chicago,UGN,42.422,-87.868,2023-02-06,55,26


# Join features

In [69]:
# (features 14-19)
user_features = users[['user_id','gender', 'married', 'has_children','age', 'tenure_months' ]].copy()

In [70]:
user_features = user_features.merge(amount_of_trips, on='user_id', how='left').merge(canceled_trips_count, on='user_id', how='left').merge(valid_session_duration, on='user_id', how='left').merge(flight_discount_amount, on='user_id', how='left').merge(hotel_discount_amount, on='user_id', how='left').merge(user_rooms, on='user_id', how='left').merge(user_nights, on='user_id', how='left').merge(user_hotel_price, on='user_id', how='left').merge(user_hotels, on='user_id', how='left').merge(user_seats, on='user_id', how='left').merge(user_flight_price, on='user_id', how='left').merge(user_flights, on='user_id', how='left').merge(user_checked_bags, on='user_id', how='left')

In [71]:
user_features

,user_id,gender,married,has_children,age,tenure_months,amount_of_trips,canceled_trips,session_duration,flight_discount_amount,hotel_discount_amount,rooms,nights,hotel_price,hotel_count,seats,flight_price,flight_count,checked_bags
0,531931,F,False,False,40,27,2,NaN,0 days 00:01:10.875000,0.1000,0.100,1.500000,3.500000,714.000000,2.0,1.500000,1676.630000,2.0,0.500000
1,611065,F,False,False,43,25,4,NaN,0 days 00:02:26.750000,0.0000,0.000,1.500000,2.250000,677.750000,4.0,1.500000,1790.872500,4.0,0.750000
2,586762,F,False,False,37,26,3,NaN,0 days 00:01:56.375000,0.1500,0.250,1.333333,7.333333,2140.666667,3.0,1.333333,762.496667,3.0,0.333333
3,229330,F,True,False,48,33,4,NaN,0 days 00:01:59.625000,0.0000,0.100,1.750000,2.000000,1028.500000,4.0,1.750000,720.802500,4.0,0.250000
4,536035,M,False,False,29,27,2,NaN,0 days 00:01:42.750000,0.2000,0.150,1.500000,1.500000,291.000000,2.0,1.500000,931.700000,2.0,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5777,591155,M,True,False,42,26,2,NaN,0 days 00:01:55.125000,0.3500,0.075,1.000000,4.000000,541.500000,2.0,1.000000,546.350000,1.0,1.000000
5778,521235,F,False,False,35,27,5,NaN,0 days 00:02:59.875000,0.1500,0.000,1.600000,5.400000,918.200000,5.0,1.800000,1541.054000,5.0,1.200000
5779,600351,F,True,False,48,26,2,NaN,0 days 00:01:52.500000,0.0000,0.250,1.000000,5.000000,1300.000000,1.0,1.000000,523.340000,2.0,0.500000
5780,577044,F,True,True,55,26,3,NaN,0 days 00:02:30.111111111,0.1000,0.050,1.000000,1.000000,270.500000,2.0,1.333333,449.743333,3.0,0.333333


In [72]:
user_features.describe()

,user_id,age,tenure_months,amount_of_trips,canceled_trips,session_duration,flight_discount_amount,hotel_discount_amount,rooms,nights,hotel_price,hotel_count,seats,flight_price,flight_count,checked_bags
count,5782.000000,5782.000000,5782.000000,5782.000000,574.000000,5782,5782.000000,5782.000000,5140.000000,5140.000000,5140.000000,5140.000000,4855.000000,4855.000000,4855.000000,4855.000000
mean,547670.236077,42.679004,26.534936,2.677966,1.026132,0 days 00:01:55.272506561,0.108658,0.074257,1.200513,4.297670,928.610853,2.600973,1.185156,677.675536,2.605973,0.579303
std,64035.394540,12.047004,1.161008,1.520225,0.159668,0 days 00:00:35.760349934,0.085947,0.069002,0.354829,2.740330,948.106056,1.277174,0.367670,1356.791234,1.284694,0.457929
min,94883.000000,18.000000,23.000000,0.000000,1.000000,0 days 00:00:30.666666666,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,8.170000,1.000000,0.000000
25%,519413.750000,36.000000,26.000000,2.000000,1.000000,0 days 00:01:30.555555555,0.050000,0.000000,1.000000,2.500000,413.500000,2.000000,1.000000,287.740000,2.000000,0.250000
50%,542279.500000,43.000000,27.000000,3.000000,1.000000,0 days 00:01:51.500000,0.100000,0.075000,1.000000,3.666667,667.000000,2.000000,1.000000,415.833333,2.000000,0.500000
75%,576215.500000,50.000000,27.000000,4.000000,1.000000,0 days 00:02:16,0.150000,0.100000,1.333333,5.250000,1094.000000,3.000000,1.250000,647.800000,3.000000,1.000000
max,844489.000000,90.000000,38.000000,8.000000,2.000000,0 days 00:07:11.333333333,0.550000,0.450000,4.000000,30.000000,14057.000000,8.000000,6.000000,41666.050000,8.000000,5.000000


In [73]:
user_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   user_id                 5782 non-null   int64          
 1   gender                  5782 non-null   object         
 2   married                 5782 non-null   bool           
 3   has_children            5782 non-null   bool           
 4   age                     5782 non-null   int64          
 5   tenure_months           5782 non-null   int64          
 6   amount_of_trips         5782 non-null   int64          
 7   canceled_trips          574 non-null    float64        
 8   session_duration        5782 non-null   timedelta64[ns]
 9   flight_discount_amount  5782 non-null   float64        
 10  hotel_discount_amount   5782 non-null   float64        
 11  rooms                   5140 non-null   float64        
 12  nights                  5140 non-n

# Handling missing values

In [74]:
user_features

,user_id,gender,married,has_children,age,tenure_months,amount_of_trips,canceled_trips,session_duration,flight_discount_amount,hotel_discount_amount,rooms,nights,hotel_price,hotel_count,seats,flight_price,flight_count,checked_bags
0,531931,F,False,False,40,27,2,NaN,0 days 00:01:10.875000,0.1000,0.100,1.500000,3.500000,714.000000,2.0,1.500000,1676.630000,2.0,0.500000
1,611065,F,False,False,43,25,4,NaN,0 days 00:02:26.750000,0.0000,0.000,1.500000,2.250000,677.750000,4.0,1.500000,1790.872500,4.0,0.750000
2,586762,F,False,False,37,26,3,NaN,0 days 00:01:56.375000,0.1500,0.250,1.333333,7.333333,2140.666667,3.0,1.333333,762.496667,3.0,0.333333
3,229330,F,True,False,48,33,4,NaN,0 days 00:01:59.625000,0.0000,0.100,1.750000,2.000000,1028.500000,4.0,1.750000,720.802500,4.0,0.250000
4,536035,M,False,False,29,27,2,NaN,0 days 00:01:42.750000,0.2000,0.150,1.500000,1.500000,291.000000,2.0,1.500000,931.700000,2.0,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5777,591155,M,True,False,42,26,2,NaN,0 days 00:01:55.125000,0.3500,0.075,1.000000,4.000000,541.500000,2.0,1.000000,546.350000,1.0,1.000000
5778,521235,F,False,False,35,27,5,NaN,0 days 00:02:59.875000,0.1500,0.000,1.600000,5.400000,918.200000,5.0,1.800000,1541.054000,5.0,1.200000
5779,600351,F,True,False,48,26,2,NaN,0 days 00:01:52.500000,0.0000,0.250,1.000000,5.000000,1300.000000,1.0,1.000000,523.340000,2.0,0.500000
5780,577044,F,True,True,55,26,3,NaN,0 days 00:02:30.111111111,0.1000,0.050,1.000000,1.000000,270.500000,2.0,1.333333,449.743333,3.0,0.333333


In [41]:
user_features.head()

,user_id,gender,married,has_children,age,tenure_months,amount_of_trips_x,canceled_trips_x,session_duration_x,flight_discount_amount_x,...,flight_discount_amount_y,hotel_discount_amount_y,rooms_y,nights_y,hotel_price_y,hotel_count_y,seats_y,flight_price_y,flight_count_y,checked_bags_y
0,531931,F,False,False,40,27,2,NaN,0 days 00:01:10.875000,0.10,...,0.10,0.10,1.500000,3.500000,714.000000,2.0,1.500000,1676.630000,2.0,0.500000
1,611065,F,False,False,43,25,4,NaN,0 days 00:02:26.750000,0.00,...,0.00,0.00,1.500000,2.250000,677.750000,4.0,1.500000,1790.872500,4.0,0.750000
2,586762,F,False,False,37,26,3,NaN,0 days 00:01:56.375000,0.15,...,0.15,0.25,1.333333,7.333333,2140.666667,3.0,1.333333,762.496667,3.0,0.333333
3,229330,F,True,False,48,33,4,NaN,0 days 00:01:59.625000,0.00,...,0.00,0.10,1.750000,2.000000,1028.500000,4.0,1.750000,720.802500,4.0,0.250000
4,536035,M,False,False,29,27,2,NaN,0 days 00:01:42.750000,0.20,...,0.20,0.15,1.500000,1.500000,291.000000,2.0,1.500000,931.700000,2.0,0.500000


In [76]:
user_features['canceled_trips'] = user_features['canceled_trips'].fillna(0)

In [77]:
user_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype          
---  ------                  --------------  -----          
 0   user_id                 5782 non-null   int64          
 1   gender                  5782 non-null   object         
 2   married                 5782 non-null   bool           
 3   has_children            5782 non-null   bool           
 4   age                     5782 non-null   int64          
 5   tenure_months           5782 non-null   int64          
 6   amount_of_trips         5782 non-null   int64          
 7   canceled_trips          5782 non-null   float64        
 8   session_duration        5782 non-null   timedelta64[ns]
 9   flight_discount_amount  5782 non-null   float64        
 10  hotel_discount_amount   5782 non-null   float64        
 11  rooms                   5140 non-null   float64        
 12  nights                  5140 non-n

In [78]:
user_features.head()

,user_id,gender,married,has_children,age,tenure_months,amount_of_trips,canceled_trips,session_duration,flight_discount_amount,hotel_discount_amount,rooms,nights,hotel_price,hotel_count,seats,flight_price,flight_count,checked_bags
0,531931,F,False,False,40,27,2,0.0,0 days 00:01:10.875000,0.10,0.10,1.500000,3.500000,714.000000,2.0,1.500000,1676.630000,2.0,0.500000
1,611065,F,False,False,43,25,4,0.0,0 days 00:02:26.750000,0.00,0.00,1.500000,2.250000,677.750000,4.0,1.500000,1790.872500,4.0,0.750000
2,586762,F,False,False,37,26,3,0.0,0 days 00:01:56.375000,0.15,0.25,1.333333,7.333333,2140.666667,3.0,1.333333,762.496667,3.0,0.333333
3,229330,F,True,False,48,33,4,0.0,0 days 00:01:59.625000,0.00,0.10,1.750000,2.000000,1028.500000,4.0,1.750000,720.802500,4.0,0.250000
4,536035,M,False,False,29,27,2,0.0,0 days 00:01:42.750000,0.20,0.15,1.500000,1.500000,291.000000,2.0,1.500000,931.700000,2.0,0.500000


In [80]:
user_features['flight_count'] = user_features['flight_count'].fillna(0)
user_features['hotel_count'] = user_features['hotel_count'].fillna(0)

In [81]:
user_features.rename(columns={'session_duration':'session_duration_avg', 'rooms':'rooms_avg', 'nights':'nights_avg', 'hotel_price':'hotel_price_avg', 'seats':'seats_avg','flight_price':'flight_price_avg', 'checked_bags':'checked_bags_avg'}, inplace=True)

In [82]:
user_features['session_duration_avg'].dt.total_seconds()

0        70.875000
1       146.750000
2       116.375000
3       119.625000
4       102.750000
           ...    
5777    115.125000
5778    179.875000
5779    112.500000
5780    150.111111
5781     83.750000
Name: session_duration_avg, Length: 5782, dtype: float64

In [83]:
user_features['session_duration_avg_sec'] = user_features['session_duration_avg'].dt.total_seconds()

In [84]:
user_features['gender'] = user_features['gender'].replace({'F': 1, 'M': 0, 'O': 2})

/var/folders/cg/_vgckz8568g7vn87wmg8bhkm0000gn/T/ipykernel_23338/3822554979.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  user_features['gender'] = user_features['gender'].replace({'F': 1, 'M': 0, 'O': 2})


In [85]:
user_features = user_features.drop(columns='session_duration_avg')

In [86]:
user_features.head()

,user_id,gender,married,has_children,age,tenure_months,amount_of_trips,canceled_trips,flight_discount_amount,hotel_discount_amount,rooms_avg,nights_avg,hotel_price_avg,hotel_count,seats_avg,flight_price_avg,flight_count,checked_bags_avg,session_duration_avg_sec
0,531931,1,False,False,40,27,2,0.0,0.10,0.10,1.500000,3.500000,714.000000,2.0,1.500000,1676.630000,2.0,0.500000,70.875
1,611065,1,False,False,43,25,4,0.0,0.00,0.00,1.500000,2.250000,677.750000,4.0,1.500000,1790.872500,4.0,0.750000,146.750
2,586762,1,False,False,37,26,3,0.0,0.15,0.25,1.333333,7.333333,2140.666667,3.0,1.333333,762.496667,3.0,0.333333,116.375
3,229330,1,True,False,48,33,4,0.0,0.00,0.10,1.750000,2.000000,1028.500000,4.0,1.750000,720.802500,4.0,0.250000,119.625
4,536035,0,False,False,29,27,2,0.0,0.20,0.15,1.500000,1.500000,291.000000,2.0,1.500000,931.700000,2.0,0.500000,102.750


In [87]:
user_features.to_csv('user_features.csv', index=False)

In [89]:
df = pd.read_csv('user_features.csv')

In [90]:
df

,user_id,gender,married,has_children,age,tenure_months,amount_of_trips,canceled_trips,flight_discount_amount,hotel_discount_amount,rooms_avg,nights_avg,hotel_price_avg,hotel_count,seats_avg,flight_price_avg,flight_count,checked_bags_avg,session_duration_avg_sec
0,531931,1,False,False,40,27,2,0.0,0.1000,0.100,1.500000,3.500000,714.000000,2.0,1.500000,1676.630000,2.0,0.500000,70.875000
1,611065,1,False,False,43,25,4,0.0,0.0000,0.000,1.500000,2.250000,677.750000,4.0,1.500000,1790.872500,4.0,0.750000,146.750000
2,586762,1,False,False,37,26,3,0.0,0.1500,0.250,1.333333,7.333333,2140.666667,3.0,1.333333,762.496667,3.0,0.333333,116.375000
3,229330,1,True,False,48,33,4,0.0,0.0000,0.100,1.750000,2.000000,1028.500000,4.0,1.750000,720.802500,4.0,0.250000,119.625000
4,536035,0,False,False,29,27,2,0.0,0.2000,0.150,1.500000,1.500000,291.000000,2.0,1.500000,931.700000,2.0,0.500000,102.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5777,591155,0,True,False,42,26,2,0.0,0.3500,0.075,1.000000,4.000000,541.500000,2.0,1.000000,546.350000,1.0,1.000000,115.125000
5778,521235,1,False,False,35,27,5,0.0,0.1500,0.000,1.600000,5.400000,918.200000,5.0,1.800000,1541.054000,5.0,1.200000,179.875000
5779,600351,1,True,False,48,26,2,0.0,0.0000,0.250,1.000000,5.000000,1300.000000,1.0,1.000000,523.340000,2.0,0.500000,112.500000
5780,577044,1,True,True,55,26,3,0.0,0.1000,0.050,1.000000,1.000000,270.500000,2.0,1.333333,449.743333,3.0,0.333333,150.111111
